In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import mobilenet_v3_small
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
import os


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = "./modified-dataset"
num_classes = 10
batch_size = 64
num_epochs = 30


In [10]:
train_transform = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.25),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

for split in ["train", "val", "test"]:
    path = os.path.join(data_dir, split)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing folder: {path}")
    else:
        print(f"✅ Found: {path}")

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=val_test_transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


✅ Found: ./modified-dataset\train
✅ Found: ./modified-dataset\val
✅ Found: ./modified-dataset\test


In [11]:
model = mobilenet_v3_small(weights="IMAGENET1K_V1")
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model = model.to(device)


In [12]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)


In [13]:
best_val_acc = 0

for epoch in range(num_epochs):
    model.train()
    train_loss, train_correct = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    train_loss /= len(train_dataset)
    train_acc = train_correct / len(train_dataset)

    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(val_dataset)
    val_acc = val_correct / len(val_dataset)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    scheduler.step()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_mobilenetv3.pth")


Epoch 1/30 Train Loss: 1.9584 Acc: 0.4129 | Val Loss: 1.4760 Acc: 0.7067
Epoch 2/30 Train Loss: 1.2562 Acc: 0.7521 | Val Loss: 0.9898 Acc: 0.8400
Epoch 3/30 Train Loss: 0.9214 Acc: 0.8550 | Val Loss: 0.8324 Acc: 0.9033
Epoch 4/30 Train Loss: 0.8252 Acc: 0.8967 | Val Loss: 0.7821 Acc: 0.9100
Epoch 5/30 Train Loss: 0.7748 Acc: 0.9133 | Val Loss: 0.7333 Acc: 0.9333
Epoch 6/30 Train Loss: 0.7381 Acc: 0.9271 | Val Loss: 0.7119 Acc: 0.9500
Epoch 7/30 Train Loss: 0.7073 Acc: 0.9442 | Val Loss: 0.6948 Acc: 0.9467
Epoch 8/30 Train Loss: 0.6952 Acc: 0.9458 | Val Loss: 0.6876 Acc: 0.9500
Epoch 9/30 Train Loss: 0.6745 Acc: 0.9537 | Val Loss: 0.6811 Acc: 0.9667
Epoch 10/30 Train Loss: 0.6599 Acc: 0.9613 | Val Loss: 0.6738 Acc: 0.9633
Epoch 11/30 Train Loss: 0.6512 Acc: 0.9637 | Val Loss: 0.6754 Acc: 0.9600
Epoch 12/30 Train Loss: 0.6334 Acc: 0.9708 | Val Loss: 0.6690 Acc: 0.9700
Epoch 13/30 Train Loss: 0.6273 Acc: 0.9721 | Val Loss: 0.6655 Acc: 0.9700
Epoch 14/30 Train Loss: 0.6204 Acc: 0.9771 | Va

In [14]:
model.load_state_dict(torch.load("best_mobilenetv3.pth"))
model.eval()

test_loss, test_correct = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * images.size(0)
        test_correct += (outputs.argmax(1) == labels).sum().item()

test_loss /= len(test_dataset)
test_acc = test_correct / len(test_dataset)

print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

Test Loss: 0.6797 | Test Acc: 0.9567


In [1]:
def predict_image(image_path, model, class_names):
    model.eval()
    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((192, 192)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)
        probs = torch.softmax(outputs, dim=1)
        conf, pred = torch.max(probs, 1)

    return class_names[pred.item()], conf.item()

In [16]:
class_names = train_dataset.classes
image_path ="./modified-dataset/test/PCB/pcb_177.jpg"
pred_class, confidence = predict_image(image_path, model, class_names)
print(f"Predicted: {pred_class} | Confidence: {confidence:.4f}")

Predicted: PCB | Confidence: 0.9255
